In [13]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
from IPython.core.display import display, HTML
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from kaggler.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from tensorflow.keras.utils import to_categorical
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [14]:
train = pd.read_csv('./open/train.csv')
train = train.drop(['index'], axis=1)
#train.fillna('NAN', inplace=True) 


test = pd.read_csv('./open/test.csv')
test = test.drop(['index'], axis=1)
#test.fillna('NAN', inplace=True)

submit = pd.read_csv('./open/sample_submission.csv')

In [15]:
def add_var(data):
    data['CODE'] = data['gender']+data['DAYS_BIRTH'].apply(str)+data['income_total'].apply(str)+data['income_type'].apply(str)

    data['P_W_E'] = data['phone'].astype(str) + "_" + data['work_phone'].astype(str) + "_" + data['email'].astype(str)
    data['gender_reality_car'] = data['gender'] + "_" + data['reality'] + "_" + data['car'] 

    return data

train = add_var(train)
test = add_var(test)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              26457 non-null  object 
 1   car                 26457 non-null  object 
 2   reality             26457 non-null  object 
 3   child_num           26457 non-null  int64  
 4   income_total        26457 non-null  float64
 5   income_type         26457 non-null  object 
 6   edu_type            26457 non-null  object 
 7   family_type         26457 non-null  object 
 8   house_type          26457 non-null  object 
 9   DAYS_BIRTH          26457 non-null  int64  
 10  DAYS_EMPLOYED       26457 non-null  int64  
 11  FLAG_MOBIL          26457 non-null  int64  
 12  work_phone          26457 non-null  int64  
 13  phone               26457 non-null  int64  
 14  email               26457 non-null  int64  
 15  occyp_type          18286 non-null  object 
 16  fami

In [17]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

print(object_col)

['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'CODE', 'P_W_E', 'gender_reality_car']


In [18]:
def occype_process(data):
    
    # 1. 경력이 없는 사람은 직업군이 결측치임
    # 2. no_work로 대체
    data['occyp_type'] = data['occyp_type'].fillna('NONE')
    data.loc[data['DAYS_EMPLOYED'] > 0, 'occyp_type'] = 'NO_WORK'
    #data.loc[(data['income_type'] == 'Student') & (data['occyp_type'] == 'NONE'), 'occyp_type'] = 'other_work'
    
    return data

train = occype_process(train)
test = occype_process(test)

In [19]:
def get_try02(data):
    
    data['gender'] = data['gender'].replace(['F','M'],[0,1])
    data['car'] = data['car'].replace(['N','Y'],[0,1])
    data['reality'] = data['reality'].replace(['N','Y'],[0,1])

    mapping_edu_type = {
    'Lower secondary' : 0, # 중학교 미만
    'Secondary / secondary special' : 1, #중학교
    'Incomplete higher' : 2, # 고등학교 중퇴
    'Higher education' : 3, # 고등학교 졸업
    'Academic degree' : 4 # 학사 이상
    }
    data['edu_type'] = data.edu_type.map(mapping_edu_type)

    return data 

In [20]:
train = get_try02(train)
test = get_try02(test)

In [21]:
from sklearn import preprocessing
def label_encode(data):
    label_encoder=preprocessing.LabelEncoder()
    data['income_type']=label_encoder.fit_transform(data['income_type'])
    data['occyp_type']=label_encoder.fit_transform(data['occyp_type'])
    data['family_type']=label_encoder.fit_transform(data['family_type'])
    data['house_type']=label_encoder.fit_transform(data['house_type'])

#     cat_cols = ['income_type', 'occyp_type', 'family_type', 'house_type']
#     lbe = LabelEncoder(min_obs=10)
#     data[cat_cols] = lbe.fit_transform(data[cat_cols])
#     data[cat_cols].head()

    return data

In [22]:
train = label_encode(train)
test = label_encode(test)

In [ ]:
#minus 변경하고
#구간화 함수
def make_bin(df, variable, n):
    data = df
    #data[variable] =- data[variable]
    count, bin_dividers = np.histogram(data[variable], bins=n)
    bin_names=[str(i) for i in range(n)]
    data['%s_bin' % variable]=pd.cut(x=data[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    #data['%s_bin' % variable] = pd.factorize(data['%s_bin' % variable])[0]

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              26457 non-null  int64  
 1   car                 26457 non-null  int64  
 2   reality             26457 non-null  int64  
 3   child_num           26457 non-null  int64  
 4   income_total        26457 non-null  float64
 5   income_type         26457 non-null  int32  
 6   edu_type            26457 non-null  int64  
 7   family_type         26457 non-null  int32  
 8   house_type          26457 non-null  int32  
 9   DAYS_BIRTH          26457 non-null  int64  
 10  DAYS_EMPLOYED       26457 non-null  int64  
 11  FLAG_MOBIL          26457 non-null  int64  
 12  work_phone          26457 non-null  int64  
 13  phone               26457 non-null  int64  
 14  email               26457 non-null  int64  
 15  occyp_type          26457 non-null  int32  
 16  fami

In [24]:
random.seed(42)
cat_models={}

train_x = train.drop(['credit'],axis=1)
train_y = train['credit']
cat_features =[0,1,2,5,6,7,8,15,18,19,20]

""
def run_kfold():
    
    folds=StratifiedKFold(n_splits=10, shuffle=True, random_state=55)
    outcomes=[]
    sub=np.zeros((test.shape[0], 3))  
    
    for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
        print(f'===================================={n_fold+1}============================================')
        X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
        y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]


        cat = CatBoostClassifier(n_estimators=1000, max_depth=8, random_state=42, langevin=True)
        cat.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_val, y_val)],
              early_stopping_rounds=35, cat_features=cat_features,
              verbose=100)
        
        cat_models[n_fold] = cat

        predictions = cat.predict_proba(X_val)
        test_predictions = cat.predict_proba(test)

        logloss=log_loss(to_categorical(y_val), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold+1} : logloss:{logloss}")
        
        sub+= test_predictions
        
        print(f'================================================================================\n\n')
        
    mean_outcome=np.mean(outcomes)
    print("Mean:{}".format(mean_outcome))
    return sub/folds.n_splits
    
my_submission = run_kfold()

====================================1============================================
Learning rate set to 0.115044
0:	learn: 1.0351787	test: 1.0351787	test1: 1.0364589	best: 1.0364589 (0)	total: 167ms	remaining: 2m 47s
100:	learn: 0.6835572	test: 0.4884555	test1: 0.6854456	best: 0.6849256 (95)	total: 12.6s	remaining: 1m 52s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6843304698
bestIteration = 116

Shrink model to first 117 iterations.
FOLD 1 : logloss:0.684330469794846


====================================2============================================
Learning rate set to 0.115044
0:	learn: 1.0353969	test: 1.0353969	test1: 1.0355156	best: 1.0355156 (0)	total: 24.9ms	remaining: 24.8s
100:	learn: 0.6877187	test: 0.4882527	test1: 0.6775011	best: 0.6775011 (100)	total: 12s	remaining: 1m 46s
200:	learn: 0.6479269	test: 0.4788739	test1: 0.6770761	best: 0.6766479 (184)	total: 25.4s	remaining: 1m 41s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6766

In [25]:
def catboost_eval(
                  depth , 
                  learning_rate ,
                  min_data_in_leaf , 
                  border_count):
    
    train_x = train.drop(['credit'],axis=1)
    train_y = train['credit']
    cat_features =[0,1,2,5,6,7,8,15,18,19,20]
    
    n_splits=5
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    predict = None
    params = {}
    params['n_estimators'] = 1000
    params['random_seed'] = 1234
    params['learning_rate'] = learning_rate
    params['min_data_in_leaf'] = int(round(min_data_in_leaf))
    params['depth'] = int(round(depth))
    params['border_count'] = int(round(border_count))
    #params['bagging_temperature'] = int(round(bagging_temperature))
    
    outcomes=[]
    
    for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
        X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
        y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]
        
        cat = CatBoostClassifier(**params , langevin=True)
        cat.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_val, y_val)],
              early_stopping_rounds=35, cat_features=cat_features,
              verbose=100)

        predictions = cat.predict_proba(X_val)

        logloss=log_loss(to_categorical(y_val), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
        print(f'================================================================================\n\n')
        
    mean_outcome=np.mean(outcomes)
    print("Mean:{}".format(mean_outcome))



In [51]:
# from bayes_opt import BayesianOptimization
# import xgboost as xgb
# import catboost as cat

# catBO = BayesianOptimization(catboost_eval,
#                              {
#                               'depth': (4, 12) ,
#                               "learning_rate" : (0.001,0.2) , 
#                               "min_data_in_leaf" : (1,100) , 
#                               'border_count': (5, 255) ,
#                              },
#                              random_state=0)
# init_round=5
# opt_round = 5
# catBO.maximize(init_points=init_round, n_iter=opt_round)

|   iter    |  target   | border... |   depth   | learni... | min_da... |
-------------------------------------------------------------------------
0:	learn: 1.0315614	test: 1.0313347	test1: 1.0318010	best: 1.0318010 (0)	total: 224ms	remaining: 3m 43s


KeyboardInterrupt: 

In [26]:
my_submission

array([[0.10458441, 0.17036093, 0.72505466],
       [0.33310086, 0.24604633, 0.42085281],
       [0.04465527, 0.08118987, 0.87415486],
       ...,
       [0.04634496, 0.11156255, 0.84209249],
       [0.1473924 , 0.317975  , 0.5346326 ],
       [0.06666702, 0.41921234, 0.51412064]])

In [27]:
submission=pd.read_csv('./open/sample_submission.csv')
submission.loc[:,1:]=my_submission
submission

,index,0,1,2
0,26457,0.104584,0.170361,0.725055
1,26458,0.333101,0.246046,0.420853
2,26459,0.044655,0.081190,0.874155
3,26460,0.066867,0.088664,0.844469
4,26461,0.088391,0.251327,0.660282
...,...,...,...,...
9995,36452,0.075105,0.230223,0.694672
9996,36453,0.132170,0.409111,0.458719
9997,36454,0.046345,0.111563,0.842092
9998,36455,0.147392,0.317975,0.534633


In [28]:
submission.to_csv('./submit/10fold_cat_6692.csv', index=False) # 0.7272812144

In [29]:
submission.head(20)

,index,0,1,2
0,26457,0.104584,0.170361,0.725055
1,26458,0.333101,0.246046,0.420853
2,26459,0.044655,0.081190,0.874155
3,26460,0.066867,0.088664,0.844469
4,26461,0.088391,0.251327,0.660282
5,26462,0.054841,0.142860,0.802299
6,26463,0.536299,0.435889,0.027812
7,26464,0.051554,0.091803,0.856643
8,26465,0.063229,0.094747,0.842023
9,26466,0.060777,0.282759,0.656464
